In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import normalize,StandardScaler
from sklearn.model_selection import cross_val_predict, cross_val_score, GridSearchCV,train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score,f1_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss,roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
Dataset = pd.read_csv("Dataset_Female_Final.csv")
display(Dataset)

,Unnamed: 0,Age,Menopause_age,Tscore_Hip_total,CRP,Cr,ALP,BUN,P,Ca,...,Tscore_Hip_neck,Zscore_Hip_neck,BMI,Pregnancy_Count,Histroy_Anticoagulant,Active_Smoking,History_Smoking,Calcium_Supplement,History_Diabetes_2,Refracture
0,0,66,50.0,-1.6,5.0,1.1,320.0,25.0,4.5,8.9,...,-2.3,-0.8,26.700000,13.0,0,0,0,0,0,0
1,1,65,50.0,-2.6,6.0,0.7,170.0,17.0,4.2,8.9,...,-3.5,-2.0,28.300000,12.0,1,0,0,0,0,0
2,2,72,40.0,-1.7,29.0,0.7,190.0,19.0,4.3,8.9,...,-3.0,-1.1,24.400000,12.0,0,0,0,0,0,0
3,3,62,40.0,-0.2,1.0,0.9,175.0,16.0,3.2,8.8,...,-1.5,0.7,22.600000,12.0,0,0,0,0,0,0
4,4,76,50.0,-1.6,13.0,1.2,290.0,23.0,3.3,8.5,...,-1.8,0.3,33.200000,10.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,707,77,50.0,-2.2,11.0,1.0,134.0,11.0,3.7,9.2,...,-2.9,-0.8,28.827532,4.0,0,0,0,0,0,1
708,708,65,50.0,-1.0,33.0,0.8,123.0,12.0,2.3,9.6,...,-2.0,-0.5,28.827532,4.0,0,0,0,0,0,1
709,709,62,50.0,-0.1,19.0,0.9,201.0,12.0,4.7,8.8,...,-2.0,-0.7,28.827532,4.0,0,0,0,0,0,0
710,710,56,50.0,0.1,32.0,1.0,153.0,16.0,4.0,8.8,...,-1.2,-0.2,28.827532,4.0,0,0,0,0,0,0


In [3]:
Dataset.drop("Unnamed: 0", axis=1, inplace = True)

In [4]:
display(Dataset)

,Age,Menopause_age,Tscore_Hip_total,CRP,Cr,ALP,BUN,P,Ca,PTH,...,Tscore_Hip_neck,Zscore_Hip_neck,BMI,Pregnancy_Count,Histroy_Anticoagulant,Active_Smoking,History_Smoking,Calcium_Supplement,History_Diabetes_2,Refracture
0,66,50.0,-1.6,5.0,1.1,320.0,25.0,4.5,8.9,35.0,...,-2.3,-0.8,26.700000,13.0,0,0,0,0,0,0
1,65,50.0,-2.6,6.0,0.7,170.0,17.0,4.2,8.9,31.4,...,-3.5,-2.0,28.300000,12.0,1,0,0,0,0,0
2,72,40.0,-1.7,29.0,0.7,190.0,19.0,4.3,8.9,42.0,...,-3.0,-1.1,24.400000,12.0,0,0,0,0,0,0
3,62,40.0,-0.2,1.0,0.9,175.0,16.0,3.2,8.8,53.0,...,-1.5,0.7,22.600000,12.0,0,0,0,0,0,0
4,76,50.0,-1.6,13.0,1.2,290.0,23.0,3.3,8.5,65.0,...,-1.8,0.3,33.200000,10.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,77,50.0,-2.2,11.0,1.0,134.0,11.0,3.7,9.2,60.7,...,-2.9,-0.8,28.827532,4.0,0,0,0,0,0,1
708,65,50.0,-1.0,33.0,0.8,123.0,12.0,2.3,9.6,48.0,...,-2.0,-0.5,28.827532,4.0,0,0,0,0,0,1
709,62,50.0,-0.1,19.0,0.9,201.0,12.0,4.7,8.8,185.0,...,-2.0,-0.7,28.827532,4.0,0,0,0,0,0,0
710,56,50.0,0.1,32.0,1.0,153.0,16.0,4.0,8.8,68.0,...,-1.2,-0.2,28.827532,4.0,0,0,0,0,0,0


In [5]:
Dataset.columns

Index(['Age', 'Menopause_age', 'Tscore_Hip_total', 'CRP', 'Cr', 'ALP', 'BUN',
       'P', 'Ca', 'PTH', 'Vit_D3', 'BMD_vertebra', 'Tscore_vertebra',
       'Zscore_vertebra', 'BMD_Hip_total', 'Zscore_hip_total', 'BMD_Hip_Neck',
       'Tscore_Hip_neck', 'Zscore_Hip_neck', 'BMI', 'Pregnancy_Count',
       'Histroy_Anticoagulant', 'Active_Smoking', 'History_Smoking',
       'Calcium_Supplement', 'History_Diabetes_2', 'Refracture'],
      dtype='object')

In [6]:
data_mean, data_std = np.mean(Dataset['Age']), np.std(Dataset['Age'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Age'] if x < lower or x > upper]
print('Age Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Age'])):
    if Dataset['Age'][i] < lower:
        Dataset['Age'][i] = lower
        
        
for i in range(len(Dataset['Age'])):
    if Dataset['Age'][i] > upper:
        Dataset['Age'][i] = upper
        

outliers = [x for x in Dataset['Age'] if x < lower or x > upper]
print('Age Column Identified outliers: %d' % len(outliers))
###########################################################################################

data_mean, data_std = np.mean(Dataset['Tscore_Hip_total']), np.std(Dataset['Tscore_Hip_total'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Tscore_Hip_total'] if x < lower or x > upper]
print('Tscore_Hip_total Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Tscore_Hip_total'])):
    if Dataset['Tscore_Hip_total'][i] < lower:
        Dataset['Tscore_Hip_total'][i] = lower
        
        
for i in range(len(Dataset['Tscore_Hip_total'])):
    if Dataset['Tscore_Hip_total'][i] > upper:
        Dataset['Tscore_Hip_total'][i] = upper
        

outliers = [x for x in Dataset['Tscore_Hip_total'] if x < lower or x > upper]
print('Tscore_Hip_total Column Identified outliers: %d' % len(outliers))
#########################################################################################

data_mean, data_std = np.mean(Dataset['CRP']), np.std(Dataset['CRP'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['CRP'] if x < lower or x > upper]
print('CRP Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['CRP'])):
    if Dataset['CRP'][i] < lower:
        Dataset['CRP'][i] = lower
        
        
for i in range(len(Dataset['CRP'])):
    if Dataset['CRP'][i] > upper:
        Dataset['CRP'][i] = upper
        

outliers = [x for x in Dataset['CRP'] if x < lower or x > upper]
print('CRP Column Identified outliers: %d' % len(outliers))
########################################################################################
data_mean, data_std = np.mean(Dataset['Cr']), np.std(Dataset['Cr'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Cr'] if x < lower or x > upper]
print('Cr Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Cr'])):
    if Dataset['Cr'][i] < lower:
        Dataset['Cr'][i] = lower
        
        
for i in range(len(Dataset['Cr'])):
    if Dataset['Cr'][i] > upper:
        Dataset['Cr'][i] = upper
        

outliers = [x for x in Dataset['Cr'] if x < lower or x > upper]
print('Cr Column Identified outliers: %d' % len(outliers))
###########################################################################################

data_mean, data_std = np.mean(Dataset['ALP']), np.std(Dataset['ALP'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['ALP'] if x < lower or x > upper]
print('ALP Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['ALP'])):
    if Dataset['ALP'][i] < lower:
        Dataset['ALP'][i] = lower
        
        
for i in range(len(Dataset['ALP'])):
    if Dataset['ALP'][i] > upper:
        Dataset['ALP'][i] = upper
        

outliers = [x for x in Dataset['ALP'] if x < lower or x > upper]
print('ALP Column Identified outliers: %d' % len(outliers))
##########################################################################################

data_mean, data_std = np.mean(Dataset['BUN']), np.std(Dataset['BUN'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['BUN'] if x < lower or x > upper]
print('BUN Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['BUN'])):
    if Dataset['BUN'][i] < lower:
        Dataset['BUN'][i] = lower
        
        
for i in range(len(Dataset['BUN'])):
    if Dataset['BUN'][i] > upper:
        Dataset['BUN'][i] = upper
        

outliers = [x for x in Dataset['BUN'] if x < lower or x > upper]
print('BUN Column Identified outliers: %d' % len(outliers))
#############################################################################################

data_mean, data_std = np.mean(Dataset['P']), np.std(Dataset['P'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['P'] if x < lower or x > upper]
print('P Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['P'])):
    if Dataset['P'][i] < lower:
        Dataset['P'][i] = lower
        
        
for i in range(len(Dataset['P'])):
    if Dataset['P'][i] > upper:
        Dataset['P'][i] = upper
        

outliers = [x for x in Dataset['P'] if x < lower or x > upper]
print('P Column Identified outliers: %d' % len(outliers))
##############################################################################################

data_mean, data_std = np.mean(Dataset['Ca']), np.std(Dataset['Ca'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Ca'] if x < lower or x > upper]
print('Ca Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Ca'])):
    if Dataset['Ca'][i] < lower:
        Dataset['Ca'][i] = lower
        
        
for i in range(len(Dataset['Ca'])):
    if Dataset['Ca'][i] > upper:
        Dataset['Ca'][i] = upper
        

outliers = [x for x in Dataset['Ca'] if x < lower or x > upper]
print('Ca Column Identified outliers: %d' % len(outliers))
############################################################################################

data_mean, data_std = np.mean(Dataset['PTH']), np.std(Dataset['PTH'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['PTH'] if x < lower or x > upper]
print('PTH Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['PTH'])):
    if Dataset['PTH'][i] < lower:
        Dataset['PTH'][i] = lower
        
        
for i in range(len(Dataset['PTH'])):
    if Dataset['PTH'][i] > upper:
        Dataset['PTH'][i] = upper
        

outliers = [x for x in Dataset['PTH'] if x < lower or x > upper]
print('PTH Column Identified outliers: %d' % len(outliers))
##########################################################################################

data_mean, data_std = np.mean(Dataset['Vit_D3']), np.std(Dataset['Vit_D3'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Vit_D3'] if x < lower or x > upper]
print('Vit_D3 Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Vit_D3'])):
    if Dataset['Vit_D3'][i] < lower:
        Dataset['Vit_D3'][i] = lower
        
        
for i in range(len(Dataset['Vit_D3'])):
    if Dataset['Vit_D3'][i] > upper:
        Dataset['Vit_D3'][i] = upper
        

outliers = [x for x in Dataset['Vit_D3'] if x < lower or x > upper]
print('Vit_D3 Column Identified outliers: %d' % len(outliers))
########################################################################################

data_mean, data_std = np.mean(Dataset['BMD_vertebra']), np.std(Dataset['BMD_vertebra'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['BMD_vertebra'] if x < lower or x > upper]
print('BMD_vertebra Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['BMD_vertebra'])):
    if Dataset['BMD_vertebra'][i] < lower:
        Dataset['BMD_vertebra'][i] = lower
        
        
for i in range(len(Dataset['BMD_vertebra'])):
    if Dataset['BMD_vertebra'][i] > upper:
        Dataset['BMD_vertebra'][i] = upper
        

outliers = [x for x in Dataset['BMD_vertebra'] if x < lower or x > upper]
print('BMD_vertebra Column Identified outliers: %d' % len(outliers))
############################################################################################

data_mean, data_std = np.mean(Dataset['Tscore_vertebra']), np.std(Dataset['Tscore_vertebra'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Tscore_vertebra'] if x < lower or x > upper]
print('Tscore_vertebra Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Tscore_vertebra'])):
    if Dataset['Tscore_vertebra'][i] < lower:
        Dataset['Tscore_vertebra'][i] = lower
        
        
for i in range(len(Dataset['Tscore_vertebra'])):
    if Dataset['Tscore_vertebra'][i] > upper:
        Dataset['Tscore_vertebra'][i] = upper
        

outliers = [x for x in Dataset['Tscore_vertebra'] if x < lower or x > upper]
print('Tscore_vertebra Column Identified outliers: %d' % len(outliers))
##########################################################################################

data_mean, data_std = np.mean(Dataset['Zscore_vertebra']), np.std(Dataset['Zscore_vertebra'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Zscore_vertebra'] if x < lower or x > upper]
print('Zscore_vertebra Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Zscore_vertebra'])):
    if Dataset['Zscore_vertebra'][i] < lower:
        Dataset['Zscore_vertebra'][i] = lower
        
        
for i in range(len(Dataset['Zscore_vertebra'])):
    if Dataset['Zscore_vertebra'][i] > upper:
        Dataset['Zscore_vertebra'][i] = upper
        

outliers = [x for x in Dataset['Zscore_vertebra'] if x < lower or x > upper]
print('Zscore_vertebra Column Identified outliers: %d' % len(outliers))
########################################################################################

data_mean, data_std = np.mean(Dataset['BMD_Hip_total']), np.std(Dataset['BMD_Hip_total'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['BMD_Hip_total'] if x < lower or x > upper]
print('BMD_Hip_total Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['BMD_Hip_total'])):
    if Dataset['BMD_Hip_total'][i] < lower:
        Dataset['BMD_Hip_total'][i] = lower
        
        
for i in range(len(Dataset['BMD_Hip_total'])):
    if Dataset['BMD_Hip_total'][i] > upper:
        Dataset['BMD_Hip_total'][i] = upper
        

outliers = [x for x in Dataset['BMD_Hip_total'] if x < lower or x > upper]
print('BMD_Hip_total Column Identified outliers: %d' % len(outliers))
############################################################################################

data_mean, data_std = np.mean(Dataset['Zscore_hip_total']), np.std(Dataset['Zscore_hip_total'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Zscore_hip_total'] if x < lower or x > upper]
print('Zscore_hip_total Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Zscore_hip_total'])):
    if Dataset['Zscore_hip_total'][i] < lower:
        Dataset['Zscore_hip_total'][i] = lower
        
        
for i in range(len(Dataset['Zscore_hip_total'])):
    if Dataset['Zscore_hip_total'][i] > upper:
        Dataset['Zscore_hip_total'][i] = upper
        

outliers = [x for x in Dataset['Zscore_hip_total'] if x < lower or x > upper]
print('Zscore_hip_total Column Identified outliers: %d' % len(outliers))
##########################################################################################

data_mean, data_std = np.mean(Dataset['BMD_Hip_Neck']), np.std(Dataset['BMD_Hip_Neck'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['BMD_Hip_Neck'] if x < lower or x > upper]
print('BMD_Hip_Neck Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['BMD_Hip_Neck'])):
    if Dataset['BMD_Hip_Neck'][i] < lower:
        Dataset['BMD_Hip_Neck'][i] = lower
        
        
for i in range(len(Dataset['BMD_Hip_Neck'])):
    if Dataset['BMD_Hip_Neck'][i] > upper:
        Dataset['BMD_Hip_Neck'][i] = upper
        

outliers = [x for x in Dataset['BMD_Hip_Neck'] if x < lower or x > upper]
print('BMD_Hip_Neck Column Identified outliers: %d' % len(outliers))
########################################################################################

data_mean, data_std = np.mean(Dataset['Tscore_Hip_neck']), np.std(Dataset['Tscore_Hip_neck'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Tscore_Hip_neck'] if x < lower or x > upper]
print('Tscore_Hip_neck Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Tscore_Hip_neck'])):
    if Dataset['Tscore_Hip_neck'][i] < lower:
        Dataset['Tscore_Hip_neck'][i] = lower
        
        
for i in range(len(Dataset['Tscore_Hip_neck'])):
    if Dataset['Tscore_Hip_neck'][i] > upper:
        Dataset['Tscore_Hip_neck'][i] = upper
        

outliers = [x for x in Dataset['Tscore_Hip_neck'] if x < lower or x > upper]
print('Tscore_Hip_neck Column Identified outliers: %d' % len(outliers))
############################################################################################

data_mean, data_std = np.mean(Dataset['Zscore_Hip_neck']), np.std(Dataset['Zscore_Hip_neck'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Zscore_Hip_neck'] if x < lower or x > upper]
print('Zscore_Hip_neck Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Zscore_Hip_neck'])):
    if Dataset['Zscore_Hip_neck'][i] < lower:
        Dataset['Zscore_Hip_neck'][i] = lower
        
        
for i in range(len(Dataset['Zscore_Hip_neck'])):
    if Dataset['Zscore_Hip_neck'][i] > upper:
        Dataset['Zscore_Hip_neck'][i] = upper
        

outliers = [x for x in Dataset['Zscore_Hip_neck'] if x < lower or x > upper]
print('Zscore_Hip_neck Column Identified outliers: %d' % len(outliers))

##########################################################################################

data_mean, data_std = np.mean(Dataset['BMI']), np.std(Dataset['BMI'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['BMI'] if x < lower or x > upper]
print('BMI Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['BMI'])):
    if Dataset['BMI'][i] < lower:
        Dataset['BMI'][i] = lower
        
        
for i in range(len(Dataset['BMI'])):
    if Dataset['BMI'][i] > upper:
        Dataset['BMI'][i] = upper
        

outliers = [x for x in Dataset['BMI'] if x < lower or x > upper]
print('BMI Column Identified outliers: %d' % len(outliers))
########################################################################################

data_mean, data_std = np.mean(Dataset['Menopause_age']), np.std(Dataset['Menopause_age'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Menopause_age'] if x < lower or x > upper]
print('Menopause_age Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Menopause_age'])):
    if Dataset['Menopause_age'][i] < lower:
        Dataset['Menopause_age'][i] = lower
        
        
for i in range(len(Dataset['Menopause_age'])):
    if Dataset['Menopause_age'][i] > upper:
        Dataset['Menopause_age'][i] = upper
        

outliers = [x for x in Dataset['Menopause_age'] if x < lower or x > upper]
print('Menopause_age Column Identified outliers: %d' % len(outliers))
###########################################################################################
data_mean, data_std = np.mean(Dataset['Pregnancy_Count']), np.std(Dataset['Pregnancy_Count'])
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in Dataset['Pregnancy_Count'] if x < lower or x > upper]
print('Pregnancy_Count Column Identified outliers: %d' % len(outliers))

for i in range(len(Dataset['Pregnancy_Count'])):
    if Dataset['Pregnancy_Count'][i] < lower:
        Dataset['Pregnancy_Count'][i] = lower
        
        
for i in range(len(Dataset['Pregnancy_Count'])):
    if Dataset['Pregnancy_Count'][i] > upper:
        Dataset['Pregnancy_Count'][i] = upper
        

outliers = [x for x in Dataset['Pregnancy_Count'] if x < lower or x > upper]
print('Pregnancy_Count Column Identified outliers: %d' % len(outliers))

Age Column Identified outliers: 1
Age Column Identified outliers: 0
Tscore_Hip_total Column Identified outliers: 3
Tscore_Hip_total Column Identified outliers: 0
CRP Column Identified outliers: 25
CRP Column Identified outliers: 0
Cr Column Identified outliers: 11
Cr Column Identified outliers: 0
ALP Column Identified outliers: 11
ALP Column Identified outliers: 0
BUN Column Identified outliers: 15
BUN Column Identified outliers: 0
P Column Identified outliers: 4
P Column Identified outliers: 0
Ca Column Identified outliers: 6
Ca Column Identified outliers: 0
PTH Column Identified outliers: 5
PTH Column Identified outliers: 0
Vit_D3 Column Identified outliers: 10
Vit_D3 Column Identified outliers: 0
BMD_vertebra Column Identified outliers: 4
BMD_vertebra Column Identified outliers: 0
Tscore_vertebra Column Identified outliers: 5
Tscore_vertebra Column Identified outliers: 0
Zscore_vertebra Column Identified outliers: 4
Zscore_vertebra Column Identified outliers: 0
BMD_Hip_total Column 

C:\Users\alish\AppData\Local\Temp\ipykernel_36872\3804175493.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset['Age'][i] = upper
C:\Users\alish\AppData\Local\Temp\ipykernel_36872\3804175493.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset['Tscore_Hip_total'][i] = lower
C:\Users\alish\AppData\Local\Temp\ipykernel_36872\3804175493.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset['Tscore_Hip_total'][i] =

In [7]:
X= Dataset.loc[:, Dataset.columns != 'Refracture']
Y = Dataset["Refracture"]

In [8]:
sm = SMOTE(random_state=5)
X_balanced, Y_balanced = sm.fit_resample(X, Y)

In [9]:
X.columns

Index(['Age', 'Menopause_age', 'Tscore_Hip_total', 'CRP', 'Cr', 'ALP', 'BUN',
       'P', 'Ca', 'PTH', 'Vit_D3', 'BMD_vertebra', 'Tscore_vertebra',
       'Zscore_vertebra', 'BMD_Hip_total', 'Zscore_hip_total', 'BMD_Hip_Neck',
       'Tscore_Hip_neck', 'Zscore_Hip_neck', 'BMI', 'Pregnancy_Count',
       'Histroy_Anticoagulant', 'Active_Smoking', 'History_Smoking',
       'Calcium_Supplement', 'History_Diabetes_2'],
      dtype='object')

In [10]:
X_balanced[['Age', 'Tscore_Hip_total','CRP',
       'Cr', 'ALP', 'BUN',
       'P', 'Ca', 'PTH'
       ,'Vit_D3', 'BMD_vertebra', 'Tscore_vertebra', 'Zscore_vertebra',
        'BMD_Hip_total', 'Zscore_hip_total', 'BMD_Hip_Neck', 'Tscore_Hip_neck',
        'Zscore_Hip_neck', 'BMI', 'Menopause_age', 'Pregnancy_Count']] = normalize(X_balanced[['Age', 'Tscore_Hip_total','CRP',
       'Cr', 'ALP', 'BUN',
       'P', 'Ca', 'PTH'
       ,'Vit_D3', 'BMD_vertebra', 'Tscore_vertebra', 'Zscore_vertebra',
        'BMD_Hip_total', 'Zscore_hip_total', 'BMD_Hip_Neck', 'Tscore_Hip_neck',
        'Zscore_Hip_neck', 'BMI', 'Menopause_age', 'Pregnancy_Count']])
X_balanced.head()

,Age,Menopause_age,Tscore_Hip_total,CRP,Cr,ALP,BUN,P,Ca,PTH,...,BMD_Hip_Neck,Tscore_Hip_neck,Zscore_Hip_neck,BMI,Pregnancy_Count,Histroy_Anticoagulant,Active_Smoking,History_Smoking,Calcium_Supplement,History_Diabetes_2
0,0.196206,0.148641,-0.004757,0.014864,0.003270,0.951300,0.074320,0.013378,0.026458,0.104048,...,0.001766,-0.006837,-0.002378,0.079374,0.030252,0,0,0,0,0
1,0.322270,0.247900,-0.012891,0.029748,0.003471,0.842859,0.084286,0.020824,0.044126,0.155681,...,0.002266,-0.017353,-0.009916,0.140311,0.050453,1,0,0,0,0
2,0.330991,0.183884,-0.007815,0.133316,0.003218,0.873450,0.087345,0.019768,0.040914,0.193078,...,0.002363,-0.013791,-0.005057,0.112169,0.046781,0,0,0,0,0
3,0.302979,0.195470,-0.000977,0.004887,0.004398,0.855183,0.078188,0.015638,0.043003,0.258998,...,0.003347,-0.007330,0.003421,0.110441,0.049728,0,0,0,0,0
4,0.239590,0.157625,-0.005044,0.040982,0.003783,0.914223,0.072507,0.010403,0.026796,0.204912,...,0.002033,-0.005674,0.000946,0.104663,0.031525,0,0,0,0,0


In [11]:
Y = Y_balanced
X.columns

Index(['Age', 'Menopause_age', 'Tscore_Hip_total', 'CRP', 'Cr', 'ALP', 'BUN',
       'P', 'Ca', 'PTH', 'Vit_D3', 'BMD_vertebra', 'Tscore_vertebra',
       'Zscore_vertebra', 'BMD_Hip_total', 'Zscore_hip_total', 'BMD_Hip_Neck',
       'Tscore_Hip_neck', 'Zscore_Hip_neck', 'BMI', 'Pregnancy_Count',
       'Histroy_Anticoagulant', 'Active_Smoking', 'History_Smoking',
       'Calcium_Supplement', 'History_Diabetes_2'],
      dtype='object')

In [12]:
def display_results(results):
    print(f'Best parameters are: {results.best_params_}')
    print(f'Best Score is : {results.best_score_} ')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
def showResults(model, modelType , X, Y):
    scores_accuracy = cross_val_score(model, X, Y, cv=10, scoring='accuracy')
    scores_log_loss = cross_val_score(model, X, Y, cv=10, scoring='neg_log_loss')
    scores_briar = cross_val_score(model, X, Y, cv=10, scoring='neg_brier_score')
    scores_auc = cross_val_score(model, X, Y, cv=10, scoring='roc_auc')
    scores_recall = cross_val_score(model, X, Y, cv=10, scoring='recall')
    scores_precision = cross_val_score(model, X, Y, cv=10, scoring='precision')
    scores_f1 = cross_val_score(model, X, Y, cv=10, scoring='f1')
    print('K-fold cross-validation results:')
    print(modelType ," average accuracy is %2.3f" % scores_accuracy.mean())
    print(modelType ," average log_loss is %2.3f" % -scores_log_loss.mean())
    print(modelType ," average brier score is %2.3f" % -scores_briar.mean())
    print(modelType ," average auc is %2.3f" % scores_auc.mean())
    print(modelType ," average recall is %2.3f" % scores_recall.mean())
    print(modelType ," average precision is %2.3f" % scores_precision.mean())
    print(modelType ," average f1 is %2.3f" % scores_f1.mean())
    
params = {
    "max_depth" : [2,4,8,16,32,None],
    'n_estimators': range(100,1000,200),
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12]
}

In [13]:
#Pysician Opinion
X = X_balanced[['Age', 'Menopause_age', 'Tscore_Hip_total', 'CRP', 'Cr', 'ALP', 'BUN',
       'P', 'Ca', 'PTH', 'Vit_D3', 'BMD_vertebra', 'Tscore_vertebra',
       'Zscore_vertebra', 'BMD_Hip_total', 'Zscore_hip_total', 'BMD_Hip_Neck',
       'Tscore_Hip_neck', 'Zscore_Hip_neck', 'BMI', 'Pregnancy_Count',
       'Histroy_Anticoagulant', 'Active_Smoking', 'History_Smoking',
       'Calcium_Supplement', 'History_Diabetes_2']]

In [14]:
rf = RandomForestClassifier(random_state=5)

cv = GridSearchCV(rf,params,cv=10, verbose=1)
cv.fit(X,Y.values.ravel())

Fitting 10 folds for each of 270 candidates, totalling 2700 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=5),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': range(100, 1000, 200)},
             verbose=1)

In [15]:
display_results(cv)

Best parameters are: {'max_depth': 32, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
Best Score is : 0.7896017699115044 


0.615 + or -0.087 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
0.613 + or -0.08 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
0.615 + or -0.086 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 500}
0.609 + or -0.087 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 700}
0.609 + or -0.084 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
0.615 + or -0.087 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
0.613 + or -0.08 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
0.615 + or -0.086 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples

In [14]:
rf = RandomForestClassifier(random_state=5, 
                            max_depth = 32, 
                            min_samples_leaf = 3, 
                            min_samples_split = 8, 
                            n_estimators = 100)
rf.fit(X,Y)

showResults(rf, "Random Forest", X, Y)

K-fold cross-validation results:
Random Forest  average accuracy is 0.790
Random Forest  average log_loss is 0.518
Random Forest  average brier score is 0.169
Random Forest  average auc is 0.867
Random Forest  average recall is 0.844
Random Forest  average precision is 0.765
Random Forest  average f1 is 0.800


In [15]:
#RFECV + Random Forest
X = X_balanced[['Age', 'Menopause_age', 'Tscore_Hip_total', 'CRP', 'Cr', 'ALP', 'BUN', 'P', 
                'Ca', 'PTH', 'Vit_D3', 'Tscore_vertebra', 'Zscore_vertebra', 'BMD_Hip_total', 
                'Zscore_hip_total', 'BMD_Hip_Neck', 'Zscore_Hip_neck', 'BMI', 'Pregnancy_Count']]

In [18]:
rf = RandomForestClassifier(random_state=5)

cv = GridSearchCV(rf,params,cv=10, verbose=1)
cv.fit(X,Y.values.ravel())

Fitting 10 folds for each of 270 candidates, totalling 2700 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=5),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': range(100, 1000, 200)},
             verbose=1)

In [19]:
display_results(cv)

Best parameters are: {'max_depth': 16, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
Best Score is : 0.7860303413400758 


0.593 + or -0.106 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
0.592 + or -0.1 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
0.597 + or -0.099 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 500}
0.599 + or -0.101 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 700}
0.597 + or -0.095 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
0.593 + or -0.106 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
0.591 + or -0.099 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
0.596 + or -0.098 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples

In [16]:
rf = RandomForestClassifier(random_state=5, 
                            max_depth = 16, 
                            min_samples_leaf = 3, 
                            min_samples_split = 8, 
                            n_estimators = 900)
rf.fit(X,Y)

showResults(rf, "Random Forest", X, Y)

K-fold cross-validation results:
Random Forest  average accuracy is 0.786
Random Forest  average log_loss is 0.509
Random Forest  average brier score is 0.165
Random Forest  average auc is 0.883
Random Forest  average recall is 0.854
Random Forest  average precision is 0.759
Random Forest  average f1 is 0.802


In [17]:
#RFECV + XGBoost
X = X_balanced[['Age', 'Menopause_age', 'CRP', 'Cr', 'ALP', 'BUN', 'P', 'Ca', 'PTH', 
                'BMD_Hip_Neck', 'Zscore_Hip_neck', 'Pregnancy_Count', 'Histroy_Anticoagulant', 
                'History_Smoking', 'History_Diabetes_2']]

In [22]:
rf = RandomForestClassifier(random_state=5)

cv = GridSearchCV(rf,params,cv=10, verbose=1)
cv.fit(X,Y.values.ravel())

Fitting 10 folds for each of 270 candidates, totalling 2700 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=5),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': range(100, 1000, 200)},
             verbose=1)

In [23]:
display_results(cv)

Best parameters are: {'max_depth': 32, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
Best Score is : 0.7904867256637168 


0.599 + or -0.1 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
0.604 + or -0.101 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
0.608 + or -0.113 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 500}
0.607 + or -0.109 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 700}
0.612 + or -0.11 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
0.599 + or -0.1 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
0.604 + or -0.101 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
0.608 + or -0.113 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_sp

In [18]:
rf = RandomForestClassifier(random_state=5, 
                            max_depth = 32, 
                            min_samples_leaf = 3, 
                            min_samples_split = 8, 
                            n_estimators = 300)
rf.fit(X,Y)

showResults(rf, "Random Forest", X, Y)

K-fold cross-validation results:
Random Forest  average accuracy is 0.790
Random Forest  average log_loss is 0.521
Random Forest  average brier score is 0.170
Random Forest  average auc is 0.862
Random Forest  average recall is 0.847
Random Forest  average precision is 0.768
Random Forest  average f1 is 0.803


In [19]:
#RFECV + CatBoost
X = X_balanced[['Age', 'Menopause_age', 'Tscore_Hip_total', 'CRP', 'Cr', 'ALP', 'BUN', 'P', 
                'Ca', 'PTH', 'Vit_D3', 'BMD_vertebra', 'Tscore_vertebra', 'Zscore_vertebra', 
                'BMD_Hip_total', 'Zscore_hip_total', 'BMD_Hip_Neck', 'Zscore_Hip_neck', 'BMI', 
                'Pregnancy_Count']]

In [26]:
rf = RandomForestClassifier(random_state=5)

cv = GridSearchCV(rf,params,cv=10, verbose=1)
cv.fit(X,Y.values.ravel())

Fitting 10 folds for each of 270 candidates, totalling 2700 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=5),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': range(100, 1000, 200)},
             verbose=1)

In [27]:
display_results(cv)

Best parameters are: {'max_depth': 32, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
Best Score is : 0.7860382427307205 


0.601 + or -0.109 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
0.585 + or -0.107 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
0.592 + or -0.101 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 500}
0.591 + or -0.101 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 700}
0.591 + or -0.101 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
0.601 + or -0.109 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
0.585 + or -0.107 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
0.592 + or -0.101 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samp

In [20]:
rf = RandomForestClassifier(random_state=5, 
                            max_depth = 32, 
                            min_samples_leaf = 3, 
                            min_samples_split = 10, 
                            n_estimators = 100)
rf.fit(X,Y)

showResults(rf, "Random Forest", X, Y)

K-fold cross-validation results:
Random Forest  average accuracy is 0.786
Random Forest  average log_loss is 0.522
Random Forest  average brier score is 0.171
Random Forest  average auc is 0.866
Random Forest  average recall is 0.844
Random Forest  average precision is 0.764
Random Forest  average f1 is 0.800


In [21]:
#RFECV + Logistic Regression
X = X_balanced[['Cr', 'BMD_Hip_total', 'Histroy_Anticoagulant', 'History_Smoking']]

In [30]:
rf = RandomForestClassifier(random_state=5)

cv = GridSearchCV(rf,params,cv=10, verbose=1)
cv.fit(X,Y.values.ravel())

Fitting 10 folds for each of 270 candidates, totalling 2700 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=5),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': range(100, 1000, 200)},
             verbose=1)

In [31]:
display_results(cv)

Best parameters are: {'max_depth': 32, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Best Score is : 0.5994469026548672 


0.574 + or -0.056 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
0.573 + or -0.055 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
0.573 + or -0.054 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 500}
0.571 + or -0.053 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 700}
0.573 + or -0.054 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
0.574 + or -0.056 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
0.573 + or -0.055 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
0.573 + or -0.054 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samp

In [22]:
rf = RandomForestClassifier(random_state=5, 
                            max_depth = 32, 
                            min_samples_leaf = 4, 
                            min_samples_split = 10, 
                            n_estimators = 100)
rf.fit(X,Y)

showResults(rf, "Random Forest", X, Y)

K-fold cross-validation results:
Random Forest  average accuracy is 0.599
Random Forest  average log_loss is 0.667
Random Forest  average brier score is 0.237
Random Forest  average auc is 0.641
Random Forest  average recall is 0.618
Random Forest  average precision is 0.599
Random Forest  average f1 is 0.607


In [23]:
#RFECV + LightGBM
X = X_balanced[['Menopause_age', 'CRP', 'ALP', 'Ca', 'BMD_Hip_Neck', 'BMI']]

In [34]:
rf = RandomForestClassifier(random_state=5)

cv = GridSearchCV(rf,params,cv=10, verbose=1)
cv.fit(X,Y.values.ravel())

Fitting 10 folds for each of 270 candidates, totalling 2700 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=5),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': range(100, 1000, 200)},
             verbose=1)

In [35]:
display_results(cv)

Best parameters are: {'max_depth': 16, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
Best Score is : 0.760287610619469 


0.583 + or -0.042 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
0.583 + or -0.035 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
0.58 + or -0.03 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 500}
0.58 + or -0.036 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 700}
0.577 + or -0.036 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
0.583 + or -0.042 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
0.583 + or -0.035 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
0.579 + or -0.03 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_sp

In [24]:
rf = RandomForestClassifier(random_state=5, 
                            max_depth = 16, 
                            min_samples_leaf = 3, 
                            min_samples_split = 8, 
                            n_estimators = 900)
rf.fit(X,Y)

showResults(rf, "Random Forest", X, Y)

K-fold cross-validation results:
Random Forest  average accuracy is 0.760
Random Forest  average log_loss is 0.536
Random Forest  average brier score is 0.178
Random Forest  average auc is 0.835
Random Forest  average recall is 0.794
Random Forest  average precision is 0.745
Random Forest  average f1 is 0.767


In [25]:
#RFECV + ADABoost
X = X_balanced[['Age', 'Menopause_age', 'Cr', 'ALP', 'Ca', 'Vit_D3', 'Tscore_vertebra', 
                'BMD_Hip_Neck', 'Pregnancy_Count']]

In [38]:
rf = RandomForestClassifier(random_state=5)

cv = GridSearchCV(rf,params,cv=10, verbose=1)
cv.fit(X,Y.values.ravel())

Fitting 10 folds for each of 270 candidates, totalling 2700 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=5),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': range(100, 1000, 200)},
             verbose=1)

In [39]:
display_results(cv)

Best parameters are: {'max_depth': 32, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
Best Score is : 0.740771175726928 


0.553 + or -0.115 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
0.56 + or -0.105 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
0.562 + or -0.103 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 500}
0.56 + or -0.101 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 700}
0.561 + or -0.104 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 900}
0.553 + or -0.115 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
0.56 + or -0.105 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
0.562 + or -0.103 for the {'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_s

In [26]:
rf = RandomForestClassifier(random_state=5, 
                            max_depth = 32, 
                            min_samples_leaf = 3, 
                            min_samples_split = 8, 
                            n_estimators = 900)
rf.fit(X,Y)

showResults(rf, "Random Forest", X, Y)

K-fold cross-validation results:
Random Forest  average accuracy is 0.741
Random Forest  average log_loss is 0.564
Random Forest  average brier score is 0.189
Random Forest  average auc is 0.812
Random Forest  average recall is 0.830
Random Forest  average precision is 0.712
Random Forest  average f1 is 0.764
